In [141]:
from maps_agent import maps_agent
from schedule_agent import Schedule_agent

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, load_tools
from langchain.tools import Tool,tool,StructuredTool
from langchain.prompts import PromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition,InjectedState
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    ToolMessage,
)
from pydantic import BaseModel, Field
import pytz
from datetime import datetime
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools.base import InjectedToolCallId
from typing_extensions import TypedDict
from typing import Annotated, Literal
#get graph visuals
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
import os
import requests
from dotenv import load_dotenv 
import geocoder
# gradio
import gradio as gr
load_dotenv()

True

In [142]:
GOOGLE_API_KEY=os.getenv('google_api_key')
pse=os.getenv('pse')
OPENWEATHERMAP_API_KEY=os.getenv('open_weather_key')
os.environ['OPENWEATHERMAP_API_KEY']=OPENWEATHERMAP_API_KEY

In [143]:
GEMINI_MODEL='gemini-2.0-flash'
llm = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model=GEMINI_MODEL, temperature=0.3)

In [144]:
class State(TypedDict):
    messages:Annotated[list, add_messages]



In [145]:
maps_ai=maps_agent(llm)
Schedule_ai=Schedule_agent(llm)

In [146]:
# extra tools for question answering

# initializing time and date tool

#creating a schema
class time_tool_schema(BaseModel):
  continent: str = Field(description='continent')
  city: str = Field(description='city')

def date_time_tool(continent: str,city: str) -> str:
  """
  tool to get the current date and time in a city.

  """
  city=city.replace(' ','_')
  continent=continent.replace(' ','_')
  query=continent+'/'+city
  timezone = pytz.timezone(query)
  # Get the current time in UTC, and then convert it to the Marrakech timezone
  utc_now = datetime.now(pytz.utc)  # Get current time in UTC
  localized_time = utc_now.astimezone(timezone)  # Convert to Marrakech time
  time=localized_time.strftime('%Y-%m-%d %H:%M:%S')
  return time

current_date_time_tool=StructuredTool.from_function(name='current_date_time_tool', func=date_time_tool, description='To get the current date and time in any city',args_schema=time_tool_schema, return_direct=True)

def google_image_search(query: str) -> str:
  """Search for images using Google Custom Search API
  args: query
  return: image url
  """
  # Define the API endpoint for Google Custom Search
  url = "https://www.googleapis.com/customsearch/v1"

  params = {
      "q": query,
      "cx": pse,
      "key": GOOGLE_API_KEY,
      "searchType": "image",  # Search for images
      "num": 1  # Number of results to fetch
  }

  # Make the request to the Google Custom Search API
  response = requests.get(url, params=params)
  data = response.json()

  # Check if the response contains image results
  if 'items' in data:
      # Extract the first image result
      image_url = data['items'][0]['link']
      return image_url
  else:
      return "Sorry, no images were found for your query."

google_image_tool=Tool(name='google_image_tool', func=google_image_search, description='Use this tool to search for images using Google Custom Search API')

In [147]:
@tool
def get_current_location_tool():
    """
    Tool to get the current location, city&continent of the user.
    agrs: none
    """
    current_location = geocoder.ip("me")
    if current_location.latlng:
        latitude, longitude = current_location.latlng
        address = current_location.address
        return f'The current location is: address:{address}, longitude:{longitude},lattitude:{latitude}.'
    else:
        return None

In [148]:
@tool
def schedule_manager(query:str):
    """
    Use this tool for any schedule related queries
    this tool can: 
    list the local files
    load a schedule
    make edits to the schedule
    answer questions about the schedule
    save the schedule
    args:query - pass the schedule related queries directly here
    """
    response=Schedule_ai.chatbot(str(query))
    return response

In [149]:
@tool
def maps_tool(query: str):
    """
    Use this tool for any maps or location related queries
    all the context is provided in the tool, simply pass the query
    this tool can:
    get the current location
    find nearby places
    find places in different locations
    show the places that have been found
    args:query - maps or location related queries
    """
    response=maps_ai.chatbot(str(query))
    return response

In [150]:
class travel_agent:
    def __init__(self,llm: any):
        self.agent=self._setup(llm)
        

    def _setup(self,llm):
        api_tools=load_tools(['openweathermap-api','wikipedia'])
        langgraph_tools=[current_date_time_tool,google_image_tool,schedule_manager,maps_tool,get_current_location_tool]+api_tools


        graph_builder = StateGraph(State)
        
        # Modification: tell the LLM which tools it can call
        llm_with_tools = llm.bind_tools(langgraph_tools)
        tool_node = ToolNode(tools=langgraph_tools)
        def chatbot(state: State):
            """ travel assistant that answers user questions about their trip.
            Depending on the request, leverage which tools to use if necessary."""
            return {"messages": [llm_with_tools.invoke(state['messages'])]}

        graph_builder.add_node("chatbot", chatbot)

        
        graph_builder.add_node("tools", tool_node)
        # Any time a tool is called, we return to the chatbot to decide the next step
        graph_builder.set_entry_point("chatbot")

        graph_builder.add_edge("tools", "chatbot")
        graph_builder.add_conditional_edges(
            "chatbot",
            tools_condition,
        )
        memory=MemorySaver()
        graph=graph_builder.compile(checkpointer=memory)
        return graph
    
    def display_graph(self):
        return display(
            Image(
                    self.agent.get_graph().draw_mermaid_png(
                        draw_method=MermaidDrawMethod.API,
                    )
                )
            )
    def get_state(self, state_val:str):
        config = {"configurable": {"thread_id": "1"}}
        return self.agent.get_state(config).values[state_val]
    
    def stream(self,input:str):
        config = {"configurable": {"thread_id": "1"}}
        input_message = HumanMessage(content=input)
        for event in self.agent.stream({"messages": [input_message]}, config, stream_mode="values"):
            event["messages"][-1].pretty_print()

    def chatbot(self,input:str):
        config = {"configurable": {"thread_id": "1"}}
        response=self.agent.invoke({'messages':HumanMessage(content=str(input))},config)
        return response['messages'][-1].content

In [153]:
travel_assistant=travel_agent(llm)

In [154]:
travel_assistant.stream(' can you load the file test.txt')

================================ Human Message =================================

 can you load the file test.txt
================================== Ai Message ==================================
Tool Calls:
  schedule_manager (2fff88fa-4696-4984-9832-5fb72fcbd481)
 Call ID: 2fff88fa-4696-4984-9832-5fb72fcbd481
  Args:
    query: load test.txt
================================= Tool Message =================================
Name: schedule_manager

OK. I have loaded the schedule from test.txt.
================================== Ai Message ==================================

OK. I have loaded the schedule from test.txt.


In [103]:
travel_assistant.get_state('messages')

[HumanMessage(content=' can you load the file test.txt', additional_kwargs={}, response_metadata={}, id='a1761dbd-99cc-458a-ae5b-f495fcce729f'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'schedule_manager', 'arguments': '{"query": "load test.txt"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-c49e5806-b633-4fa0-a86b-fafc42864bc3-0', tool_calls=[{'name': 'schedule_manager', 'args': {'query': 'load test.txt'}, 'id': '237ad540-3719-42e1-b381-2a895646407e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 290, 'output_tokens': 8, 'total_tokens': 298, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='OK. I have loaded the schedule from test.txt.', name='schedule_manager', id='24e8d4e7-5d49-487a-8a68-41c7ba5cec76', tool_call_id='237ad540-3719-42e1-b381-2a895646407e'),
 AIMessage(content='OK. I have loaded the schedule from test.txt.', additional_kwarg

In [130]:
travel_assistant.get_state('Schedule_ai')

KeyError: 'Schedule_ai'

In [110]:
maps_ai.get_state('messages')

[HumanMessage(content='fun vegetarian restaurants in east village nyc', additional_kwargs={}, response_metadata={}, id='da735fec-f7c3-4767-a620-01af3463d91a'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'look_for_places', 'arguments': '{"query": "fun%20vegetarian%20restaurants%20in%20east%20village%20nyc"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-34fe37d5-174b-48d2-871c-3e35e58a3ac7-0', tool_calls=[{'name': 'look_for_places', 'args': {'query': 'fun%20vegetarian%20restaurants%20in%20east%20village%20nyc'}, 'id': '9fb2fadc-2b6d-4af8-a844-446a3a9a38ac', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1792, 'output_tokens': 31, 'total_tokens': 1823, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='I found 16 places', name='look_for_places', id='c4368de2-1d2d-4f09-995c-b056e0548edd', tool_call_id='9fb2fadc-2b6d-4af8-a844-446a3a9a38ac'),
 AIMess

In [105]:
def chatbot(input, history):
    response=travel_assistant.chatbot(input)
    return response

In [107]:
demo = gr.ChatInterface(chatbot, type="messages", autofocus=False)

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
